In [6]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
nltk.download('punkt')
import csv
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dkras\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
# things we need for Tensorflow
import numpy as np
import glob
import os
import tflearn
import tensorflow as tf
import random



In [8]:
# import our chat-bot intents file
import json
with open('data/intents.json') as json_data:
    intents = json.load(json_data)

files = glob.glob('models/*') #delete the models folder content
for f in files:
    os.remove(f)

In [9]:
words = []
classes = []
documents = []
ignore_words = ['dane']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

43 documents
15 classes ['Alex', 'Dan', 'Easter Eggs', 'Goodbye', 'Hello', 'Jude', 'Michal', 'Nik', 'Photos', 'Raph', 'Sasha', 'Sophie', 'bot disrespect', 'kacper', 'max']
44 unique stemmed words ['alex', 'alexand', 'cya', 'dan', 'daniel', 'danieli', 'droid', 'east', 'eg', 'fuck', 'goodby', 'goodnight', 'hello', 'hi', 'joob', 'jood', 'jud', 'kacp', 'kol', 'korneluk', 'krasovsk', 'kuchenm', 'leg', 'malyshev', 'max', 'maxim', 'maximil', 'mich', 'minimil', 'nik', 'nikolay', 'nostalg', 'ofeimu', 'off', 'on', 'piekarsk', 'raph', 'raphael', 'sash', 'slash', 'soph', 'stfu', 'you', 'zaharov']


In [10]:
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

C:\Users\dkras\AppData\Local\Temp/ipykernel_50348/1818506589.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [11]:
# reset underlying graph data
tf.compat.v1.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('models/model.tflearn')

Training Step: 5999  | total loss: 0.00954 | time: 0.009s
| Adam | epoch: 1000 | loss: 0.00954 - acc: 0.9999 -- iter: 40/43
Training Step: 6000  | total loss: 0.00908 | time: 0.010s
| Adam | epoch: 1000 | loss: 0.00908 - acc: 0.9999 -- iter: 43/43
--
INFO:tensorflow:C:\Users\dkras\OneDrive\Documents\GitHub\Discord_AI_Bot\models\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [12]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "models/training_data", "wb" ) )


model.load('models/model.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\dkras\OneDrive\Documents\GitHub\Discord_AI_Bot\models\model.tflearn


In [13]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return np.array(bag)


model.load('models/model.tflearn')


# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.5
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i, r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list


def response(sentence, userID='123', show_details=True):
    global answer_list
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                            (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print('tag:', i['tag'])
                        # a random response from the intent
                        answer = i['responses']
                        answer = ', '.join(map(str, answer))
                        answer_list = []
                        with open(answer, 'r', encoding="utf-8") as file:
                            reader = csv.reader(file)
                            k = 0
                            for row in reader:
                                answer_list.insert(k, row)
                                k += 1
                    response_type = i['type']
                    response_type = ''.join(response_type)
                    answer = random.choice(answer_list)
                    answer = ''.join(map(str, answer))
                    answer = answer.replace("{", "").replace("}", "")

                    return answer, response_type


            results.pop(0)

INFO:tensorflow:Restoring parameters from C:\Users\dkras\OneDrive\Documents\GitHub\Discord_AI_Bot\models\model.tflearn


In [14]:
classify("dane")

[('Dan', 0.98820764)]

In [15]:
response("dane")

tag: Dan


('🗿 🚭', 'Emoji')